# Preprocessing Step (Etapa de Pré-Processamento)
**[EN-US]**

Data pre-processing stage and then we feed this data to the model.

In this step we perform:
* Transformation of all words to lowercase;
* Removal of punctuations;
* Removal of stopwords;
* We apply stemming;
* Tokenization;
* Padding.

**[PT-BR]**

Etapa do pré-processamento dos dados para depois, alimentarmos esses dados ao modelo.

Nesta etapa realizamos:
* Transformação de todas as palavras para minúsculas;
* Remoção de pontuações;
* Remoção de stopwords;
* Aplicamos o stemming;
* Tokenização;
* Padding.

## Table of Contents
* [Packages](#1)
* [Preprocessing](#2)
    * [RNN Preprocessing](#2.1)
        * [Lowercasing, Stopwords, Stemming and Punctuations](#2.1.1)
        * [Tokenization](#2.1.2)
    * [Transformer Preprocessing](#2.2)

<a name="1"></a>
## Packages (Pacotes)
**[EN-US]**

Packages used in the system.

* [pandas](https://pandas.pydata.org/): is the main package for data manipulation;
* [numpy](www.numpy.org): is the main package for scientific computing;
* [re](https://docs.python.org/3/library/re.html): provides regular expression matching operations similar to those found in Perl;
* [string](https://docs.python.org/pt-br/3.13/library/string.html): for common string operations;
* [nltk](https://www.nltk.org/): NLTK is a leading platform for building Python programs to work with human language data;
* [tensorflow](https://www.tensorflow.org/): framework that makes it easy to create ML models that can run in any environment;
* [scikit-learn](https://scikit-learn.org/stable/): open source machine learning library;
* [transformers](https://huggingface.co/docs/transformers/index): provides APIs and tools to easily download and train state-of-the-art pretrained models;
* [pickle](https://docs.python.org/3/library/pickle.html): implements binary protocols for serializing and de-serializing a Python object structure;
* [os](https://docs.python.org/3/library/os.html): built-in module, provides a portable way of using operating system dependent functionality;
* [sys](https://docs.python.org/3/library/sys.html): provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter;
* [src](../src/): package with all the codes for all utility functions created for this system.

**[PT-BR]**

Pacotes utilizados no sistema.
* [pandas](https://pandas.pydata.org/): é o principal pacote para manipulação de dados;
* [numpy](www.numpy.org): é o principal pacote para computação científica;
* [re](https://docs.python.org/3/library/re.html): fornece operações de correspondência de expressões regulares semelhantes às encontradas em Perl;
* [string](https://docs.python.org/pt-br/3.13/library/string.html): para operações comuns de strings;
* [nltk](https://www.nltk.org/): NLTK é uma plataforma líder para a construção de programas Python para trabalhar com dados de linguagem humana;
* [tensorflow](https://www.tensorflow.org/): framework que facilita a criação de modelos de machine learning que podem ser executados em qualquer ambiente;
* [scikit-learn](https://scikit-learn.org/stable/): biblioteca open-source de machine learning;
* [transformers](https://huggingface.co/docs/transformers/index): fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração;
* [pickle](https://docs.python.org/3/library/pickle.html): implementa protocolos binários para serializar e desserializar uma estrutura de objeto Python;
* [os](https://docs.python.org/3/library/os.html): módulo integrado, fornece uma maneira portátil de usar funcionalidades dependentes do sistema operacional;
* [sys](https://docs.python.org/3/library/sys.html): fornece acesso a algumas variáveis usadas ou mantidas pelo interpretador e a funções que interagem fortemente com o interpretador;
* [src](../src/): pacote com todos os códigos de todas as funções utilitárias criadas para esse sistema.

In [77]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import DistilBertTokenizer
import pickle

import os
import sys
PROJECT_ROOT = os.path.abspath( # Getting Obtaining the absolute normalized version of the project root path (Obtendo a versão absoluta normalizada do path raíz do projeto)
    os.path.join( # Concatenating the paths (Concatenando os paths)
        os.getcwd(), # # Getting the path of the notebooks directory (Obtendo o path do diretório dos notebooks)
        os.pardir # Gettin the constant string used by the OS to refer to the parent directory (Obtendo a string constante usada pelo OS para fazer referência ao diretório pai)
    )
)
# Adding path to the list of strings that specify the search path for modules
# Adicionando o path à lista de strings que especifica o path de pesquisa para os módulos
sys.path.append(PROJECT_ROOT)
from src.preprocessing import *

**[EN-US]**

> **Note**: the codes for the utility functions used in this system are in the `preprocessing.py` script within the `../src/` directory.

**[PT-BR]**

> **Nota**: os códigos para as funções utilitárias utilizadas nesse sistema estão no script `preprocessing.py` dentro do diretório `../src/`.

<a name="2"></a>
## Preprocessing (Pré-processamento)
**[EN-US]**

We will create 2 models for this project. Therefore, we will do 2 different preprocessings, one for each model:
1. The first will be using an RNN with bidirectional LSTMs created and trained from scratch.
2. The second model will be the application of fine-tuning to a model with pre-trained transformers architecture.

Reading the dataset that will be pre-processed and plotting your first 5 examples.

**[PT-BR]**

Vamos criar 2 modelos para esse projeto. Portanto, faremos 2 pré-processamentos diferentes, um para cada modelo:
1. O primeiro será utilizando uma RNN com LSTMs bidirecionais criadas e treinada do zero.
2. O segundo modelo será a aplicação de fine-tuning em um modelo com arquitetura transformers pré-treinado.

Lendo o dataset que será pré-processado e projetando os seus primeiros 5 exemplos.

In [80]:
comics_data = pd.read_csv('../data/raw/comics_corpus.csv')
comics_data.head()

,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...,non-action
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...,non-action
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...,non-action


**[EN-US]**

We can see some duplicate phrases in the `description` feature. But if we explore more closely, we can see that what doesn't make them 100% similar are the scores. Therefore, let's treat the scores to eliminate duplicate examples.

**[PT-BR]**

Podemos ver algumas frases duplicadas na feature `description`. Mas se explorarmos mais de perto, podemos ver que o que não faz elas se tornarem 100% similares, são as pontuações. Portanto, vamos tratar as pontuações para eliminar os exemplos duplicados.

In [82]:
print(f'Some example:\n{comics_data["description"][1]}\n')
print(f'Duplicated example:\n{comics_data["description"][2]}')

Some example:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn’t Hel’s only ruler—and now she’s upset the cosmic balance. There will be a price to pay…and Karnilla intends to ensure the Valkyries pay it.

Duplicated example:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn?t Hel?s only ruler?and now she?s upset the cosmic balance. There will be a price to pay?and Karnilla intends to ensure the Valkyries pay it.


<a name="2.1"></a>
### RNN Preprocessing (Pré-processamento RNN)
**[EN-US]**

For a sentiment classifier, we first pre-process the raw data, then tokenize our train set and extract useful features to train our model and make our predictions.

Plotting the stopwords in English and the punctuations that will be removed.

**[PT-BR]**

Para um classificador de sentimentos, primeiro pré-processamos os dados brutos, depois, tokenizamos o nosso train set e extraímos as features úteis para treinarmos o nosso modelo e fazermos nossas previsões.

Plotando as stopwords em ingês e as pontuações que serão removidas.

In [84]:
stopwords_en = stopwords.words('english')
punct = string.punctuation
print(f'Stopwords:\n{stopwords_en}\n\nPunctuations:\n{punct}')

Stopwords:
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so'

<a name="2.1.1"></a>
#### Lowercasing, Stopwords, Stemming and Punctuations (Lowercasing, Stopwords, Stemming e Pontuações)
**[EN-US]**

After the initial preprocessing of the data, we end up with only the words that contain all the relevant information about the text.

Initial preprocessing before tokenization:
* `Lowercasing`: to reduce our vocabulary even further, without losing valuable information, we will have to put each of our words in lowercase. Therefore, the word CHILDREN, Children and children, will be treated as being exactly the same word children.
* `Special characters`: such as mathematical symbols, currency symbols, section and paragraph signs, inline markup signs and so on. It's usually safe to delete them.
* `Stopwords and punctuation`: we remove all words that do not add significant meaning to the texts, also known as stopwords and punctuation marks. Once eliminated, the general meaning of the sentence can be inferred without any effort.
* `Stemming`: is simply transforming any word into its base stem, which we can define as the set of characters used to build the word and its derivatives. The word works for example, its stem is `work`, because, adding the letter "s", it forms the word works, adding the suffix "e", forms the word worke, and adding the suffix "ing", forms the word working.
    * After performing stemming on our corpus, the words works, worke and working will be reduced to the stem `work`. Therefore, our vocabulary will be significantly reduced by carrying out this process for each word in the corpus.

Preprocessing the data, counting the initially preprocessed and duplicate examples, and plotting the first 5 examples of the initially preprocessed dataset.

**[PT-BR]**

Após o pré-processamento inicial dos dados, que acabamos apenas com as palavras que contêm todas as informações relevantes sobre o texto.

Pré-processamentos iniciais antes da tokenização:
* `Lowercasing`: para reduzir nosso vocabulário ainda mais, sem perder informações valiosas, teremos que colocar cada uma de nossas palavras em lowercase. Portanto, a palavra CHILDREN, Children e children, serão tratadas como sendo exatamente a mesma palavra children.
* `Caracteres especiais`: como símbolos matemáticos, símbolos monetários, sinais de seção e parágrafo, sinais de marcação inline e assim por diante. Geralmente é seguro excluí-los.
* `Stopwords e pontuação`: removemos todas as palavras que não acrescentam significado significativo aos textos, também conhecidas como stopwords e sinais de pontuação (punctuation marks). Após eliminadas, o significado geral da frase pode ser inferido sem nenhum esforço.
* `Stemming`: é simplesmente transformar qualquer palavra em sua base stem (raiz base), que podemos definir como o set de caracteres usados para construir a palavra e seus derivados. A palavra works por exemplo, seu stem é `work`, porque, adicionando a letra "s", forma a palavra works, adicionando o sufixo "e", forma a palavra worke, e adicionando o sufixo "ing", forma a palavra working.
    * Depois de executar a stemming no nosso corpus, as palavras works, worke e working, serão reduzidas para a stem `work`. Portanto, nosso vocabulário será reduzido significativamente ao realizar esse processo para cada palavra do corpus.

Pré-processando os dados, contando os exemplos inicialmente pré-processados e duplicados, e plotando os primeiros 5 primeiros exemplos do dataset inicialmente pré-processado.

In [86]:
comics_data['description'] = comics_data['description'].map(rnn_preprocess)
print(f'Number of duplicate examples: {comics_data["description"].duplicated().sum()}')
comics_data.head()

Number of duplicate examples: 790


,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),shadow kirisaki mountain secret histori come l...,non-action
1,93339,The Mighty Valkyries (2021) #3,children afterlif kraven hunter stalk jane fos...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),children afterlif kraven hunter stalk jane fos...,action
3,93350,X-Corp (2021) #2,shark water x corp shock debut got fenc mend h...,non-action
4,94896,X-Corp (2021) #2 (Variant),shark water x corp shock debut got fenc mend h...,non-action


**[EN-US]**

Now we can remove the duplicate examples, because they are initially pre-processed and unscored.

Plotting the number of duplicates after removing duplicates and plotting the first 5 examples from the pre-processed dataset and without duplicate examples.

**[PT-BR]**

Agora podemos remover os exemplos duplicados, porque estão inicialmente pré-processados e sem pontuações.

Plotando o número de duplicatas após a removação das duplicatas e plotando os primeiros 5 exemplos do dataset pré-processado e sem exemplos duplicados.

In [88]:
comics_data = comics_data.drop_duplicates('description')
print(f'Number of duplicate examples: {comics_data["description"].duplicated().sum()}')
comics_data.head()

Number of duplicate examples: 0


,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),shadow kirisaki mountain secret histori come l...,non-action
1,93339,The Mighty Valkyries (2021) #3,children afterlif kraven hunter stalk jane fos...,action
3,93350,X-Corp (2021) #2,shark water x corp shock debut got fenc mend h...,non-action
5,93645,Heroes Reborn: Weapon X & Final Flight (2021) #1,best world without aveng squadron suprem prote...,non-action
6,93052,Heroes Reborn (2021) #6,eon fabl daughter utopia isl known power princ...,non-action


**[EN-US]**

Creating a dataset with only the features that will be used in the model, transforming the target label $y$ into binary and plotting the first 5 examples of the dataset

**[PT-BR]**

Criando um dataset com apenas as features que serão usadas no modelo, transformando o target label $y$ em binário e plotando os primeiros 5 exemplos do dataset.

In [90]:
# Setting the new dataset (Definindo o novo dataset)
comics_corpus = comics_data[['description', 'y']].copy()
# Transforming the target label y into binary (Transformando o target label y em binário)
comics_corpus['y'] = comics_corpus['y'].map(lambda x: 1 if x == 'action' else 0)
comics_corpus.head()

,description,y
0,shadow kirisaki mountain secret histori come l...,0
1,children afterlif kraven hunter stalk jane fos...,1
3,shark water x corp shock debut got fenc mend h...,0
5,best world without aveng squadron suprem prote...,0
6,eon fabl daughter utopia isl known power princ...,0


**[EN-US]**

Dividing the dataset between training, validation and testing subsets. We use the split `stratified sampling` to try to maintain the same proportion of labels in the division between each subset.

Dividing the dataset and plotting the dimension of each of them.

**[PT-BR]**

Dividindo o dataset entre os subsets de treinamento, de validação e de teste. Usamos o split `stratified sampling (amostragem estratificada)` para tentarmos manter a mesma proporção dos labels na divisão entre cada subset.

Dividindo o dataset e plotando a dimensão de cada um deles.

In [92]:
# Splitting between training and the validation and testing subset
# Dividindo entre treinamento e o subset da validação e teste
split_train = StratifiedShuffleSplit(n_splits=1, test_size=.4, random_state=42)
for train_index, subset_index in split_train.split(comics_corpus, comics_corpus['y']):
    train_corpus, subset_corpus = comics_corpus.iloc[train_index, :], comics_corpus.iloc[subset_index, :]

# Splitting between validation and testing
# Dividindo entre validação e teste
split_test = StratifiedShuffleSplit(n_splits=1, test_size=.5, random_state=42)
for valid_index, test_index in split_test.split(subset_corpus, subset_corpus['y']):
    valid_corpus, test_corpus = subset_corpus.iloc[valid_index, :], subset_corpus.iloc[test_index, :]

print(f'Train set shape: {train_corpus.shape}\nValidation set shape: {valid_corpus.shape}\nTest set shape: {test_corpus.shape}')

Train set shape: (9682, 2)
Validation set shape: (3227, 2)
Test set shape: (3228, 2)


Setting the global variables `VOCAB_SIZE` and `MAX_LEN` to tokenize the training set (Definindo as variáveis globais `VOCAB_SIZE` e `MAX_LEN` para tokenizar o training set).

In [94]:
VOCAB_SIZE = 10000
MAX_LEN = max([len(sentence.split()) for sentence in train_corpus['description']])
print(f'Length of the largest clean sentence: {MAX_LEN}')

Length of the largest clean sentence: 166


<a name="2.1.2"></a>
#### Tokenization (Tokenização)
**[EN-US]**

In this step, we encode our training set corpus in its vector representation, that is, first we need to create a $V$ vocabulary that allows us to encode any text as a vector of integers, for example. Where our vocabulary $V$ will be a vector of unique words from our vector of texts, where we go through each word of each text and save in the vocabulary all the new words that appear in our search. Then, we map, that is, we replace each word found in the training set with its index in the vocabulary.
| Token | Index |
| :---: | :---: |
| afterlif | 1 |
| $\vdots$ | $\vdots$ |
| balanc | 615 |
| $\vdots$ | $\vdots$ |
| cosmic | 621 |
| $\vdots$ | $\vdots$ |
| work | VOCAB_SIZE |

The RNN can receive as input a vector representation of tokenized sentences and then we apply an embedding layer that will transform our tokenized representation of integers into values ​​that represent the semantics of that word numerically. This addresses one of the problems, the word order, alphabetical order, in this example, does not make much sense from a semantic point of view. For example, there is no reason 'cosmic' should be given a higher number than 'afterlif'.
> I will talk more about embeddings when creating the model.

This RNN will allow us to predict sentiment in complex sentences, which we would not be able to correctly classify using simpler methods like Naive Bayes because they miss important information.

Our representation $X$ will be a vector of integers, that is, the index of each token in our vocabulary. Once we have all the vector representations of our sentences, we need to identify the maximum vector size and pad each vector with 0 to match that size, this process is called `padding` and it ensures that all of our vectors are the same size, even if our sentences are not.

To create vocabulary, we define which words belong to the vocabulary. This means that we create a list of the words that we are going to use in our representations. One way to create this vocabulary is to look at our training set, and find the `VOCAB_SIZE` words with the most occurrence, for example, or we use already created dictionaries that tell us the `VOCAB_SIZE` words most commonly used in the language of our task.

In some tasks such as speech recognition, or question answering, we will only find and generate words from a fixed set of words, for example, a chatbot can only answer limited sets of questions. This fixed list of words is also called `closed vocabulary`. However, using a fixed set of words is not always sufficient for the task. Often, we need to deal with words that we have never seen before, which results in an `open vocabulary`. **Open vocabulary** simply means that we can find words outside of vocabulary, like the name of a new city in the training set.

If we train a neural network on a corpus of texts based on our vocabulary, when we want to make inference with the trained model, we will need to encode the text we want to infer with the same vocabulary. Otherwise it won't make sense, because the words would map to different numbers, different tokens. Qualquer palavra no corpus de treino que não esteja no vocabulário será substituída por `<UNK>`. Unknown words are also called `Out of vocabulary (OOV)`. Uma forma de lidar com palavras OOV é modelá-las com uma palavra especial, **\<UNK>**. Para fazermos isso, substituímos todas as palavras OOV por \<UNK>, o special token, `<UNK>`. The proportion of unknown words in the test set is called `OOV rate`.

At the end, we apply padding, add 0 to the end of each sequence and make them all the same length. This is called `post-padding`, because the padding tokens are at the end of the sequences.

Tensorflow and Keras offer us several ways to tokenize words. One of them is the layer I'm using in this system, `tensorflow.keras.layers.TextVectorization()`.
* `tf.keras.layers.TextVectorization()`: will generate the vocabulary and create vectors from the sentences. It removes punctuations, that is, it will manage the tokens, transforming the sentences into a list of integers (the indices of each token in the vocabulary) and so on. 
* `adapt()`: method from the TextVectorization() layer, which takes the data and generates a vocabulary from the words found in these sentences.

Training the tokenizer on the training set with the previously defined `VOCAB_SIZE` and `MAX_LEN` and projecting the vocabulary size.

**[PT-BR]**

Nessa etapa, codificamos o nosso corpus do training set em sua representação vetorial, ou seja, primeiro precisamos criar um vocabulário $V$ que nos permite codificar qualquer texto como um vetor de inteiros, por exemplo. Onde o nosso vocabulário $V$ será um vetor de palavras exclusivas do nosso vetor de textos, onde percorremos cada palavra de cada texto e salvamos no vocabulário todas as novas palavras que aparecerem em nossa pesquisa. Então, mapeamos, ou seja, substituímos cada palavra encontrada no training set pelo seu índice no vocabulário.
| Token | Índice |
| :---: | :----: |
| afterlif | 1 |
| $\vdots$ | $\vdots$ |
| balanc | 615 |
| $\vdots$ | $\vdots$ |
| cosmic | 621 |
| $\vdots$ | $\vdots$ |
| work | VOCAB_SIZE |

A RNN pode receber como input uma representação vetorial de frases tokenizadas e, então, aplicamos uma embedding layer que transformará nossa representação tokenizada de inteiros em valores que representam a semântica dessa palavra numéricamente. Isso trata um dos problemas, a ordem das palavras, ordem alfabética, neste exemplo, não faz muito sentido do ponto de vista semântico. Por exemplo, não há razão para que o 'cosmic' receba um número maior do que o 'afterlif'.
> Falarei mais sobre embeddings na criação do modelo.

Essa RNN nos permitirá prever sentimentos em frases complexas, que não conseguiríamos classificar corretamente usando métodos mais simples, como Naive Bayes, porque eles perdem informações importantes.

A nossa representação $X$ será um vetor de números inteiros, ou seja, o índice de cada token do nosso vocabulário. Depois de obtermos todas as representações vetoriais das nossas frases, precisamos identificar o tamanho máximo do vetor e preencher cada vetor com 0 para corresponder a essa tamanho, esse processo é chamado de `padding` e garante que todos os nossos vetores tenham o mesmo tamanho, mesmo que nossas frases não tenham.

Para a criação do vocabulário, definimos quais palavras pertencem ao vocabulário. Isso significa que criamos uma lista com as palavras que vamos usar nas nossas representações. Uma forma de criar esse vocabulário é olhar para o nosso training set, e encontrar as `VOCAB_SIZE` palavras com mais ocorrência, por exemplo, ou utilizamos dicionários já criados que nos digam as `VOCAB_SIZE` palavras mais comumente usadas na língua da nossa tarefa.

Em algumas tarefas como speech recognition, ou question aswering, encontraremos e geraremos somente palavras a partir de um set fixo de palavras, por exemplo, um chatbot só pode responder em sets limitados de perguntas. Essa lista fixa de palavras também é chamada de `closed vocabulary`. No entanto, usar um set fixo de palavras nem sempre é suficiente para a tarefa. Muitas vezes, precisamos lidar com palavras que nunca vimos antes, o que resulta em um `open vocabulary`. **Open vocabulary** significa simplesmente que podemos encontrar palavras out of vocabulary (fora do vocabulário), como o nome de uma nova cidade no training set.

Se treinarmos uma rede neural em um corpus de textos com base nesse nosso vocabulário, quando quisermos fazer inferência com o modelo treinado, precisaremos codificar o texto que desejamos inferir com o mesmo vocabulário. Caso contrário, não fará sentido, porque as palavras seriam mapeadas para números diferentes, tokens diferentes. Qualquer palavra no corpus de treino que não esteja no vocabulário será substituída por `<UNK>`. As unknown words também são chamadas de `Out of vocabulary (OOV)`. Uma forma de lidar com palavras OOV é modelá-las com uma palavra especial, **\<UNK>**. Para fazermos isso, substituímos todas as palavras OOV por \<UNK>, o special token, `<UNK>`. A proporção de unknown words no test set é cahamda de `OOV rate`.

No final, aplicamos o `padding`, adicionamos 0 ao final de cada sequence e fazemos com que todas tenham o mesmo comprimento. Isso é chamado de `post-padding (pós-padding)`, porque os tokens de padding estão no final das sequências.

O tensorflow e o Keras nos oferecem várias maneiras de tokenizar palavras. Uma delas é a layer que estou usando nesse sistema, `tensorflow.keras.layers.TextVectorization()`.
* `tf.keras.layers.TextVectorization()`: gerará o vocabulário e criará vetores a partir das frases. Ela retira pontuações, ou seja, ela irá gerenciar os tokens, transformando as frases em uma lista de inteiros (os índices de cada token no vocabulário) e etc. 
* `adapt()`: método da layer TextVectorization(), que pega os dados e gera um vocabulário a partir das palavras encontradas nessas frases.

Treinando o tokenizador no training set com o `VOCAB_SIZE` e `MAX_LEN` definidos anteriormente e projetando o tamanho do vocabulário.

In [96]:
sentence_vec = rnn_tokenizer(train_corpus['description'], max_tokens=VOCAB_SIZE, max_len=MAX_LEN)
print(f'Vocabulary size: {sentence_vec.vocabulary_size()}')

Vocabulary size: 10000


**[EN-US]**

Applying the trained tokenizer to each subset and plotting its dimensions and the first tokenized example of the training set.

**[PT-BR]**

Aplicando o tokenizador treinado em cada subset e plotando as suas dimensões e o primeiro exemplo tokenizado do training set.

In [98]:
train_tokenized = sentence_vec(train_corpus['description'])
valid_tokenized = sentence_vec(valid_corpus['description'])
test_tokenized = sentence_vec(test_corpus['description'])
print(f'Tokenized and padded train sequences shape: {train_tokenized.shape}\n\nFirst training padded sequence:\n{train_tokenized[0]}\n')
print(f'Tokenized and padded valid sequences shape: {valid_tokenized.shape}\nTokenized and padded test sequences shape: {test_tokenized.shape}')

Tokenized and padded train sequences shape: (9682, 166)

First training padded sequence:
[ 202  101   80   45  332  101   80  804  323  767 6056 3215  815  649
  499  795  303   92  409  624    3 1593 1843 1086 2697  413    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]

Tokenized and padded valid sequences shape: (3227, 166)
Tokenized a

**[EN-US]**

Loading the trained tokenization model into the `../models/` directory for later use. We save the hyperparameters that were used in training and the generated vocabulary.

**[PT-BR]**

Carregando o modelo de tokenização treinado no diretório `../models/` para usarmos posteriormente. Salvamos os hiperparâmetros que foram utilizados no treinamento e o vocabulário gerado.

In [100]:
pickle.dump(
    {'config': sentence_vec.get_config(), 'vocabulary': sentence_vec.get_vocabulary()},
    open('../models/vectorizer.pkl', 'wb')
)

**[EN-US]**

Transforming the $y$ labels into a column vector, concatenating each tokenized corpus with the corresponding $y$ labels and plotting the dimension of each subset.

**[PT-BR]**

Transformando os labels $y$ em um vetor de coluna, concatenando cada corpus tokenizado com os labels $y$ correspondentes e plotando a dimensão de cada subset.

In [102]:
# Transforming the y labels into a column vector
# Transformando os labels y em um vetor de coluna
labels = comics_corpus['y'].to_numpy().reshape(-1, 1)

# Concatenating the corpus of each subset and the corresponding labels
# Concatenando o corpus de cada subset e os labels correspondentes
train_tokens = np.concatenate([train_tokenized, labels[train_index]], axis=1)
valid_tokens = np.concatenate([valid_tokenized, labels[valid_index]], axis=1)
test_tokens = np.concatenate([test_tokenized, labels[test_index]], axis=1)
print(f'Preprocessed train set shape: {train_tokens.shape}\nPreprocessed validation set shape: {valid_tokens.shape}\nPreprocessed test set shape: {test_tokens.shape}')

Preprocessed train set shape: (9682, 167)
Preprocessed validation set shape: (3227, 167)
Preprocessed test set shape: (3228, 167)


Loading each preprocessed dataset into the `../data/preprocessed/` directory (Carregando cada dataset pré-processado no diretório `../data/preprocessed/`).

In [122]:
# Loading the dataset with initial pre-processing to disk
# Carregando no disco o dataset com pré-processamento inicial
comics_corpus.to_csv('../data/preprocessed/comics_corpus.csv', index=False)

# Loading tokenized datasets to disk
# Carregando no disco os datasets tokenizados
np.save('../data/preprocessed/train_tokens.npy', train_tokens)
np.save('../data/preprocessed/valid_tokens.npy', valid_tokens)
np.save('../data/preprocessed/test_tokens.npy', test_tokens)

<a name="2.2"></a>
### Transformer Preprocessing (Pré-processamento RNN)
**[EN-US]**

For transformers pre-processing, we will use a pre-trained model to apply the initial pre-processing and apply tokenization and padding,

DistilBERT is a small, fast, cheap and light Transformer model trained by distilling Bert base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of Bert’s performances as measured on the GLUE (General Language Understanding Evaluation) benchmark.

[Hugging Face](https://huggingface.co/) (🤗) is the best resource for pre-trained transformers. Their open-source libraries simplify downloading, fine-tuning and using transformer models like DeepSeek, BERT, Llama, T5, Qwen, GPT-2 and much more. And the best part, you can use them alongside either TensorFlow, PyTorch or Flax. In this notebook, I use 🤗 transformers to use the `DistilBERT` model for sentiment classification. For the pre-processing step, we use the pre-trained DistilBERT tokenizer `distilbert-base-uncased-finetuned-sst-2-english`, to do this, we initialize the DistilBertTokenizer class and define the desired pre-trained model.
> In the fine-tuning of the model, in the notebook `05_transformers_finetuning.ipynb`, I will talk in more detail about tranfers learning and fine-tuning.

Tokenizing, padding the corpus and returning the tokenized corpus as pytorch tensors using the pre-trained `DistilBERT` tokenizer, setting and plotting the vector representations of the corpus.

**[PT-BR]**

Para o pré-processamento do transformers, vamos utilizar um modelo pré-treinado para aplicar os pré-processamentos iniciais e, aplicar a tokenização e o padding,

DistilBERT é um modelo Transformer pequeno, rápido, barato e leve treinado pela destilação do modelo base Bert. Ele tem 40% menos parâmetros que o bert-base-uncased, roda 60% mais rápido e preserva mais de 95% do desempenho do Bert conforme medido no benchmark GLUE (General Language Understanding Evaluation).

[Hugging Face](https://huggingface.co/) (🤗) é o melhor recurso para transformadores pré-treinados. Suas bibliotecas de código aberto simplificam o download, o ajuste fino e o uso de modelos de transformadores como DeepSeek, BERT, Llama, T5, Qwen, GPT-2 e muito mais. E a melhor parte, você pode usá-los junto com TensorFlow, PyTorch ou Flax. Neste notebook, utilizo transformadores 🤗 para usar o modelo `DistilBERT` para classificação de sentimento. Para a etapa do pré-processamento, usamos o tokenizador do DistilBERT `distilbert-base-uncased-finetuned-sst-2-english` pré-treinado, para isso inicializamos a classe DistilBertTokenizer e definidos o modelo pré-treinado desejado.
> No fine-tuning do modelo, no notebook `05_transformers_finetuning.ipynb`, falarei com mais detalhes sobre o tranfers learning e o fine-tuning.

Tokenizando, aplicando o padding no corpus e retornando o corpus tokenizado como tensores pytorch utilizando o tokenizer `DistilBERT` pré-treinado, definindo e plotando as representações vetoriais do corpus.

In [106]:
# Defining the pre-trained model
# Definindo o modelo pré-treinado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
# Defining the tokenizer
# Definindo o tokenizer
comics_transformers = tokenizer(
    comics_data['description'].tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
)
# Accessing the vector representations of the corpus
# Acessando as representações vetoriais do corpus
transformers_tokens = comics_transformers['input_ids']
transformers_tokens

tensor([[  101,  5192, 11382,  ...,     0,     0,     0],
        [  101,  2336,  2044,  ...,     0,     0,     0],
        [  101, 11420,  2300,  ...,     0,     0,     0],
        ...,
        [  101,  2010,  4263,  ...,     0,     0,     0],
        [  101,  2028,  2707,  ...,     0,     0,     0],
        [  101,  9530, 20464,  ...,     0,     0,     0]])

Concatenating the tokenized corpus with the corresponding $y$ labels and plotting the dataset dimension (Concatenando o corpus tokenizado com os labels $y$ correspondentes e plotando a dimensão do dataset).

In [108]:
dataset_transformers = np.concatenate(
    [transformers_tokens.numpy(), labels], axis=1
)
print(f'Preprocessed transformers dataset shape: {dataset_transformers.shape}')

Preprocessed transformers dataset shape: (16137, 252)


**[EN-US]**

Selecting each subset with their respective indices resulting from the `stratified sampling split` and plotting its dimensions.

**[PT-BR]**

Selecionando cada subset com os seus respectivos índices resultantes da `stratified sampling split` e plotando as suas dimensões.

In [110]:
train_transformers, valid_transformers, test_transformers = dataset_transformers[train_index], dataset_transformers[valid_index], dataset_transformers[test_index]
print(f'Train transformers set shape: {train_transformers.shape}\nValidation transformers set shape: {valid_transformers.shape}\nTest transformers set shape: {test_transformers.shape}')

Train transformers set shape: (9682, 252)
Validation transformers set shape: (3227, 252)
Test transformers set shape: (3228, 252)


Loading each preprocessed dataset into the `../data/preprocessed/` directory (Carregando cada dataset pré-processado no diretório `../data/preprocessed/`).

In [112]:
np.save('../data/preprocessed/train_transformers.npy', train_transformers)
np.save('../data/preprocessed/valid_transformers.npy', valid_transformers)
np.save('../data/preprocessed/test_transformers.npy', test_transformers)